<a href="https://colab.research.google.com/github/LucianoRodriguez0764/ipadapter_faceid_colab/blob/main/Ipadapter_faceID_Latest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing dependencies and download the IP-Adapter model

In [1]:
!pip install insightface
!pip install onnxruntime
!pip install diffusers
!pip install git+https://github.com/tencent-ailab/IP-Adapter.git
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 12.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 79.5 MB/s eta 0:00:00
  Created wheel for insightface: filename=insightface-0.7.3-cp310-cp310-linux_x86_64.whl size=1055395 sha256=4fffcf1dbfcad7b61cd1c148144937aa27376370b98dbc96d3b92e0747d6d96e
  Stored in directory: /root/.cache/pip/wheels/e3/d0/80/e3773fb8b6d1cca87ea1d33d9b1f20a223a6493c896da249b5
Successfully built insightface
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 107.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.6 MB/s eta 0:00:00
  Cloning https://github.com/tencent-ailab/IP-Adapter.git to /tmp/pip-req-build-xexavzhn
  Running command git clone --filter=blob:none --quiet https://git

# Generate the image with IP-Adapter

In [2]:
!wget -O ip-adapter-faceid-plusv2_sd15.bin https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sd15.bin

--2024-11-22 22:38:29--  https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sd15.bin
Resolving huggingface.co (huggingface.co)... 18.164.174.23, 18.164.174.55, 18.164.174.118, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.23|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/36/ca/36ca54ff1895000df817b45a5ae71fccd14170b0843d8adc2ba7944d9ac903e9/26d0d86a1d60d6cc811d3b8862178b461e1eeb651e6fe2b72ba17aa95411e313?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27ip-adapter-faceid-plusv2_sd15.bin%3B+filename%3D%22ip-adapter-faceid-plusv2_sd15.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1732574309&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMjU3NDMwOX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzM2L2NhLzM2Y2E1NGZmMTg5NTAwMGRmODE3YjQ1YTVhZTcxZmNjZDE0MTcwYjA4NDNkOGFkYzJiYTc5NDRkOWFjOTAzZTkv

In [8]:
import torch
from diffusers import StableDiffusionPipeline, DDIMScheduler, AutoencoderKL
from PIL import Image as PILImage

from ip_adapter.ip_adapter_faceid import IPAdapterFaceIDPlus

v2 = True
base_model_path = "SG161222/Realistic_Vision_V4.0_noVAE"
vae_model_path = "stabilityai/sd-vae-ft-mse"
image_encoder_path = "laion/CLIP-ViT-H-14-laion2B-s32B-b79K"
ip_ckpt = "ip-adapter-faceid-plus_sd15.bin" if not v2 else "ip-adapter-faceid-plusv2_sd15.bin"
device = "cuda"

noise_scheduler = DDIMScheduler(
    num_train_timesteps=1000,
    beta_start=0.00085,
    beta_end=0.012,
    beta_schedule="scaled_linear",
    clip_sample=False,
    set_alpha_to_one=False,
    steps_offset=1,
)
vae = AutoencoderKL.from_pretrained(vae_model_path).to(dtype=torch.float16)
pipe = StableDiffusionPipeline.from_pretrained(
    base_model_path,
    torch_dtype=torch.float16,
    scheduler=noise_scheduler,
    vae=vae,
    feature_extractor=None,
    safety_checker=None
)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

...

text_encoder/config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

tokenizer/merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

In [3]:
import cv2
from insightface.app import FaceAnalysis
from insightface.utils import face_align
import torch
import glob, os
from PIL import Image as PILImage

app = FaceAnalysis(name="buffalo_l", providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
app.prepare(ctx_id=0, det_size=(640, 640))

image_path = "images/face"
imgs = glob.glob(os.path.join(image_path, '*.*'))
#imgs += glob.glob(os.path.join('images/face_2', '*.*'))

# Collect all embeddings
embeddings = []

for i in range(len(imgs)):
    print(imgs[i])
    image = cv2.imread(imgs[i])
    faces = app.get(image)

    # Check if a face is detected
    if faces:
        faceid_embed = torch.from_numpy(faces[0].normed_embedding).unsqueeze(0)
        embeddings.append(faceid_embed)
    else:
        print("face not detected")

# Calculate the average embedding for the face ID

weights = []
weights = [elem/sum(weights) for elem in weights]
#print(weights)

if embeddings:

    ### weights
    total = torch.zeros_like(embeddings[0])
    for i in range(len(embeddings)):
        if len(weights)==len(embeddings):
            total.add_(embeddings[i]*weights[i])
        else:
            total.add_(embeddings[i]*(1/len(embeddings)))
    faceid_embeds = total / len(embeddings[i])

    ###  mean weights
    faceid_embeds_mean = torch.mean(torch.stack(embeddings), dim=0)

    print("Collected face ID embedding")
else:
    print("No faces detected in the images provided.")



# face
image1 = cv2.imread("momo.jpg")
faces1 = app.get(image1)
if faces1:
  print("person detected")
else:
  print("person face not detected")

face_image1 = face_align.norm_crop(image1, landmark=faces1[0].kps, image_size=336)

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


download_path: /root/.insightface/models/buffalo_l


100%|██████████| 281857/281857 [00:05<00:00, 51061.77KB/s]
/usr/local/lib/python3.10/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:115: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)
momo/4624126

/usr/local/lib/python3.10/dist-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


momo/} (2).jpg
momo/463997068_18373574476110237_5525011736067819438_n.jpg
momo/463611148_18373143598110237_489938125011790652_n.jpg
momo/463806231_18373143628110237_4215921533474557615_n.jpg
momo/463986561_18373574521110237_5729348757702524634_n (2).jpg
Collected face ID embedding
person detected


In [18]:
from IPython.display import display
from ipywidgets import HBox, Image
import random

def display_images(images, format="png"):
  if format=="png" or True:
    image_widgets = [Image(value=img._repr_png_()) for img in images]
  # Display the images in a row
  display(HBox(image_widgets))

prompt = ""
prompt += "30 years old man, face shot, wearing an astronaut suit"
extra_prompt = ""

negative_prompt = "(deformed iris, deformed pupils, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime), text, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, nfsw, extra legs, fused fingers, too many fingers, long neck"
negative_prompt += ""

width = 512
height = 512
num_inference_steps = 50
guidance_scale = 3
num_samples = 1
random_seed = 4574673546
use_hardcoded_seed = False
v2=True

x_plot_generation = True
y_plot_generation = False
x_plot_parameter = 'guidance_scale'
x_plot_values = [2,3,4,5]
y_plot_parameter = 'num_inference_steps'
y_plot_values = [30,40,50]
xy_different_seeds = False

ip_model = IPAdapterFaceIDPlus(pipe, image_encoder_path, ip_ckpt, device)

if not use_hardcoded_seed:
  random_seed = random.randint(1, 9999999)

if x_plot_generation:
  num_samples=1
  if y_plot_generation:
    images_array = []
    for h in range(len(y_plot_values)):
      images = []
      if y_plot_parameter in globals():
        globals()[y_plot_parameter] = y_plot_values[h]
        print(y_plot_parameter,"set to:",y_plot_values[h])
      for i in range(len(x_plot_values)):
        if x_plot_parameter in globals():
          globals()[x_plot_parameter] = x_plot_values[i]
          print(x_plot_parameter,"set to:",x_plot_values[i])
        if xy_different_seeds and i!=0:
          random_seed = random.randint(1, 9999999)
        print("Seed: ", random_seed)
        generated_image = ip_model.generate(
      prompt=prompt+", "+extra_prompt, negative_prompt=negative_prompt, face_image=face_image1, faceid_embeds=faceid_embeds_mean, num_samples=num_samples, width=width, height=height, num_inference_steps=num_inference_steps, seed=random_seed,
      guidance_scale=guidance_scale, shortcut=True, s_scale=1,
  )[0]
        images.append(generated_image)
      images_array.append(images)
    for img in images_array:
      display_images(img)
  elif x_plot_generation:
    images = []
    for i in range(len(x_plot_values)):
      if x_plot_parameter in globals():
        globals()[x_plot_parameter] = x_plot_values[i]
        print(x_plot_parameter,"set to:",x_plot_values[i])
      if xy_different_seeds and i!=0:
        random_seed = random.randint(1, 9999999)
      print("Seed: ", random_seed)
      images.extend(ip_model.generate(
      prompt=prompt+", "+extra_prompt, negative_prompt=negative_prompt, face_image=face_image1, faceid_embeds=faceid_embeds_mean, num_samples=num_samples, width=width, height=height, num_inference_steps=num_inference_steps, seed=random_seed,
      guidance_scale=guidance_scale, shortcut=True, s_scale=1,
  ))
    display_images(images)


else:

  print("Seed: ", random_seed)
  images = ip_model.generate(
      prompt=prompt+", "+extra_prompt, negative_prompt=negative_prompt, face_image=face_image1, faceid_embeds=faceid_embeds_mean, num_samples=num_samples, width=width, height=height, num_inference_steps=num_inference_steps, seed=random_seed,
      guidance_scale=guidance_scale, shortcut=True, s_scale=1,
  )

  display_images(images)

guidance_scale set to: 2
Seed:  1817195


  0%|          | 0/50 [00:00<?, ?it/s]

guidance_scale set to: 3
Seed:  1817195


  0%|          | 0/50 [00:00<?, ?it/s]

guidance_scale set to: 4
Seed:  1817195


  0%|          | 0/50 [00:00<?, ?it/s]

guidance_scale set to: 5
Seed:  1817195


  0%|          | 0/50 [00:00<?, ?it/s]